# Credit Risk Modeling - Model Training

**Dataset:** Lending Club Loan Data (Processed)

**Objective:** Train, compare, and select the best model for predicting loan default with proper handling of class imbalance.

---

## Table of Contents

1. [Setup & Data Loading](#1-setup--data-loading)
2. [Baseline Model: Logistic Regression](#2-baseline-model-logistic-regression)
3. [Advanced Model: Random Forest](#3-advanced-model-random-forest)
4. [Advanced Model: XGBoost](#4-advanced-model-xgboost)
5. [Hyperparameter Tuning](#5-hyperparameter-tuning)
6. [Model Comparison](#6-model-comparison)
7. [Best Model Selection](#7-best-model-selection)
8. [Model Persistence](#8-model-persistence)
9. [Summary](#9-summary)

---
## 1. Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import joblib
import time

# ML libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Model evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix,
    classification_report, roc_curve, precision_recall_curve
)

# Imbalanced data handling
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Visualization settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("✓ Libraries imported successfully!")

In [ ]:
# Load processed data
processed_dir = Path('../data/processed')

print("Loading processed datasets...\n")

train_df = pd.read_csv(processed_dir / 'train_data.csv')
test_df = pd.read_csv(processed_dir / 'test_data.csv')

print(f"✓ Train data loaded: {train_df.shape}")
print(f"✓ Test data loaded: {test_df.shape}")

# Load feature names
with open(processed_dir / 'feature_names.txt', 'r') as f:
    feature_names = [line.strip() for line in f.readlines()]

print(f"\n✓ Features loaded: {len(feature_names)}")

In [ ]:
# Prepare X and y
# Remove non-feature columns
cols_to_drop = ['target', 'issue_d', 'earliest_cr_line']
cols_to_drop = [col for col in cols_to_drop if col in train_df.columns]

X_train = train_df.drop(columns=cols_to_drop)
y_train = train_df['target']

X_test = test_df.drop(columns=cols_to_drop)
y_test = test_df['target']

print("Dataset Split:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"\nTrain default rate: {y_train.mean()*100:.2f}%")
print(f"Test default rate: {y_test.mean()*100:.2f}%")
print(f"\nClass imbalance ratio: {(1-y_train.mean())/y_train.mean():.1f}:1")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train set
y_train.value_counts().plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Train Set - Target Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Target (0=Paid, 1=Default)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Fully Paid', 'Default'], rotation=0)

# Test set
y_test.value_counts().plot(kind='bar', ax=axes[1], color=['green', 'red'])
axes[1].set_title('Test Set - Target Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Target (0=Paid, 1=Default)')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(['Fully Paid', 'Default'], rotation=0)

plt.tight_layout()
plt.show()

---
## 2. Baseline Model: Logistic Regression

Start with a simple, interpretable baseline model.

In [ ]:
print("Training Baseline Model: Logistic Regression\n")
print("="*70)

# Train with class weights to handle imbalance
start_time = time.time()

lr_model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

lr_model.fit(X_train, y_train)

train_time = time.time() - start_time

print(f"✓ Model trained in {train_time:.2f} seconds")

In [ ]:
# Predictions
y_train_pred_lr = lr_model.predict(X_train)
y_test_pred_lr = lr_model.predict(X_test)

y_train_proba_lr = lr_model.predict_proba(X_train)[:, 1]
y_test_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Evaluate
print("\nLogistic Regression - Performance Metrics:")
print("="*70)

print("\nTRAIN SET:")
print(f"Accuracy:  {accuracy_score(y_train, y_train_pred_lr):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_lr):.4f}")
print(f"Recall:    {recall_score(y_train, y_train_pred_lr):.4f}")
print(f"F1-Score:  {f1_score(y_train, y_train_pred_lr):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_train, y_train_proba_lr):.4f}")
print(f"PR-AUC:    {average_precision_score(y_train, y_train_proba_lr):.4f}")

print("\nTEST SET:")
print(f"Accuracy:  {accuracy_score(y_test, y_test_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_lr):.4f}")
print(f"Recall:    {recall_score(y_test, y_test_pred_lr):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_test_pred_lr):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_test_proba_lr):.4f}")
print(f"PR-AUC:    {average_precision_score(y_test, y_test_proba_lr):.4f}")

In [ ]:
# Confusion Matrix
cm_lr = confusion_matrix(y_test, y_test_pred_lr)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_title('Logistic Regression - Confusion Matrix', fontsize=14, fontweight='bold')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_xticklabels(['Paid', 'Default'])
ax.set_yticklabels(['Paid', 'Default'])
plt.tight_layout()
plt.show()

# Interpret confusion matrix
tn, fp, fn, tp = cm_lr.ravel()
print("\nConfusion Matrix Breakdown:")
print(f"True Negatives (Correctly predicted Paid):    {tn:,}")
print(f"False Positives (Predicted Default, was Paid): {fp:,}")
print(f"False Negatives (Predicted Paid, was Default): {fn:,}")
print(f"True Positives (Correctly predicted Default):  {tp:,}")

---
## 3. Advanced Model: Random Forest

In [ ]:
print("Training Random Forest Model\n")
print("="*70)

start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=100,
    min_samples_leaf=50,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_model.fit(X_train, y_train)

train_time = time.time() - start_time

print(f"✓ Model trained in {train_time:.2f} seconds")

In [ ]:
# Predictions
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

y_train_proba_rf = rf_model.predict_proba(X_train)[:, 1]
y_test_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate
print("\nRandom Forest - Performance Metrics:")
print("="*70)

print("\nTRAIN SET:")
print(f"Accuracy:  {accuracy_score(y_train, y_train_pred_rf):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_rf):.4f}")
print(f"Recall:    {recall_score(y_train, y_train_pred_rf):.4f}")
print(f"F1-Score:  {f1_score(y_train, y_train_pred_rf):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_train, y_train_proba_rf):.4f}")
print(f"PR-AUC:    {average_precision_score(y_train, y_train_proba_rf):.4f}")

print("\nTEST SET:")
print(f"Accuracy:  {accuracy_score(y_test, y_test_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_rf):.4f}")
print(f"Recall:    {recall_score(y_test, y_test_pred_rf):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_test_pred_rf):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_test_proba_rf):.4f}")
print(f"PR-AUC:    {average_precision_score(y_test, y_test_proba_rf):.4f}")

In [ ]:
# Feature Importance
feature_importance_rf = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features (Random Forest):\n")
print(feature_importance_rf.head(20))

# Visualize top 15
fig, ax = plt.subplots(figsize=(10, 8))
top_15 = feature_importance_rf.head(15)
ax.barh(range(len(top_15)), top_15['importance'].values, color='steelblue')
ax.set_yticks(range(len(top_15)))
ax.set_yticklabels(top_15['feature'].values)
ax.set_xlabel('Importance')
ax.set_title('Top 15 Feature Importances - Random Forest', fontsize=14, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

---
## 4. Advanced Model: XGBoost

In [ ]:
print("Training XGBoost Model\n")
print("="*70)

# Calculate scale_pos_weight for class imbalance
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}\n")

start_time = time.time()

xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=10,
    scale_pos_weight=scale_pos_weight,
    objective='binary:logistic',
    eval_metric='auc',
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

train_time = time.time() - start_time

print(f"✓ Model trained in {train_time:.2f} seconds")

In [ ]:
# Predictions
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

y_train_proba_xgb = xgb_model.predict_proba(X_train)[:, 1]
y_test_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluate
print("\nXGBoost - Performance Metrics:")
print("="*70)

print("\nTRAIN SET:")
print(f"Accuracy:  {accuracy_score(y_train, y_train_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_xgb):.4f}")
print(f"Recall:    {recall_score(y_train, y_train_pred_xgb):.4f}")
print(f"F1-Score:  {f1_score(y_train, y_train_pred_xgb):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_train, y_train_proba_xgb):.4f}")
print(f"PR-AUC:    {average_precision_score(y_train, y_train_proba_xgb):.4f}")

print("\nTEST SET:")
print(f"Accuracy:  {accuracy_score(y_test, y_test_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_xgb):.4f}")
print(f"Recall:    {recall_score(y_test, y_test_pred_xgb):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_test_pred_xgb):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_test_proba_xgb):.4f}")
print(f"PR-AUC:    {average_precision_score(y_test, y_test_proba_xgb):.4f}")

In [ ]:
# Feature Importance
feature_importance_xgb = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features (XGBoost):\n")
print(feature_importance_xgb.head(20))

# Visualize top 15
fig, ax = plt.subplots(figsize=(10, 8))
top_15 = feature_importance_xgb.head(15)
ax.barh(range(len(top_15)), top_15['importance'].values, color='coral')
ax.set_yticks(range(len(top_15)))
ax.set_yticklabels(top_15['feature'].values)
ax.set_xlabel('Importance')
ax.set_title('Top 15 Feature Importances - XGBoost', fontsize=14, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

---
## 5. Hyperparameter Tuning

Optimize the best performing model (typically XGBoost).

In [ ]:
print("Hyperparameter Tuning for XGBoost\n")
print("="*70)
print("Note: This may take 10-20 minutes...\n")

# Define parameter grid (reduced for faster execution)
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200],
    'min_child_weight': [5, 10]
}

# Use a smaller subset for faster tuning (optional)
# sample_size = 50000
# X_train_sample = X_train.sample(n=sample_size, random_state=42)
# y_train_sample = y_train.loc[X_train_sample.index]

# Or use full training set
X_train_sample = X_train
y_train_sample = y_train

# Grid search
xgb_base = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

grid_search = GridSearchCV(
    xgb_base,
    param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

start_time = time.time()
grid_search.fit(X_train_sample, y_train_sample)
tuning_time = time.time() - start_time

print(f"\n✓ Hyperparameter tuning completed in {tuning_time/60:.2f} minutes")
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV ROC-AUC: {grid_search.best_score_:.4f}")

In [ ]:
# Train final model with best parameters
print("\nTraining final XGBoost model with optimized parameters...\n")

xgb_tuned = grid_search.best_estimator_

# If used sample, retrain on full data
if len(X_train_sample) < len(X_train):
    xgb_tuned.fit(X_train, y_train)
    print("✓ Model retrained on full training set")

# Predictions
y_test_pred_tuned = xgb_tuned.predict(X_test)
y_test_proba_tuned = xgb_tuned.predict_proba(X_test)[:, 1]

# Evaluate
print("\nXGBoost (Tuned) - Performance Metrics:")
print("="*70)
print(f"\nTEST SET:")
print(f"Accuracy:  {accuracy_score(y_test, y_test_pred_tuned):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_tuned):.4f}")
print(f"Recall:    {recall_score(y_test, y_test_pred_tuned):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_test_pred_tuned):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_test_proba_tuned):.4f}")
print(f"PR-AUC:    {average_precision_score(y_test, y_test_proba_tuned):.4f}")

---
## 6. Model Comparison

In [ ]:
# Comprehensive model comparison
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', 'XGBoost (Tuned)'],
    'Accuracy': [
        accuracy_score(y_test, y_test_pred_lr),
        accuracy_score(y_test, y_test_pred_rf),
        accuracy_score(y_test, y_test_pred_xgb),
        accuracy_score(y_test, y_test_pred_tuned)
    ],
    'Precision': [
        precision_score(y_test, y_test_pred_lr),
        precision_score(y_test, y_test_pred_rf),
        precision_score(y_test, y_test_pred_xgb),
        precision_score(y_test, y_test_pred_tuned)
    ],
    'Recall': [
        recall_score(y_test, y_test_pred_lr),
        recall_score(y_test, y_test_pred_rf),
        recall_score(y_test, y_test_pred_xgb),
        recall_score(y_test, y_test_pred_tuned)
    ],
    'F1-Score': [
        f1_score(y_test, y_test_pred_lr),
        f1_score(y_test, y_test_pred_rf),
        f1_score(y_test, y_test_pred_xgb),
        f1_score(y_test, y_test_pred_tuned)
    ],
    'ROC-AUC': [
        roc_auc_score(y_test, y_test_proba_lr),
        roc_auc_score(y_test, y_test_proba_rf),
        roc_auc_score(y_test, y_test_proba_xgb),
        roc_auc_score(y_test, y_test_proba_tuned)
    ],
    'PR-AUC': [
        average_precision_score(y_test, y_test_proba_lr),
        average_precision_score(y_test, y_test_proba_rf),
        average_precision_score(y_test, y_test_proba_xgb),
        average_precision_score(y_test, y_test_proba_tuned)
    ]
})

print("\nModel Comparison - Test Set Performance:")
print("="*70)
print(results.to_string(index=False))

# Identify best model
best_model_idx = results['ROC-AUC'].idxmax()
print(f"\n🏆 Best Model: {results.loc[best_model_idx, 'Model']}")
print(f"   ROC-AUC: {results.loc[best_model_idx, 'ROC-AUC']:.4f}")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'PR-AUC']

for idx, metric in enumerate(metrics):
    axes[idx].bar(results['Model'], results[metric], color='steelblue', alpha=0.7)
    axes[idx].set_title(metric, fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Score')
    axes[idx].set_ylim(0, 1)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(axis='y', alpha=0.3)

plt.suptitle('Model Comparison - All Metrics', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves Comparison
fig, ax = plt.subplots(figsize=(10, 8))

models_roc = [
    ('Logistic Regression', y_test_proba_lr),
    ('Random Forest', y_test_proba_rf),
    ('XGBoost', y_test_proba_xgb),
    ('XGBoost (Tuned)', y_test_proba_tuned)
]

colors = ['blue', 'green', 'orange', 'red']

for (name, proba), color in zip(models_roc, colors):
    fpr, tpr, _ = roc_curve(y_test, proba)
    auc = roc_auc_score(y_test, proba)
    ax.plot(fpr, tpr, label=f'{name} (AUC={auc:.4f})', color=color, linewidth=2)

ax.plot([0, 1], [0, 1], 'k--', label='Random (AUC=0.5000)')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Precision-Recall Curves Comparison
fig, ax = plt.subplots(figsize=(10, 8))

for (name, proba), color in zip(models_roc, colors):
    precision, recall, _ = precision_recall_curve(y_test, proba)
    pr_auc = average_precision_score(y_test, proba)
    ax.plot(recall, precision, label=f'{name} (PR-AUC={pr_auc:.4f})', color=color, linewidth=2)

# Baseline (random classifier)
baseline = y_test.mean()
ax.axhline(y=baseline, color='k', linestyle='--', label=f'Baseline (No Skill={baseline:.4f})')

ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curves - Model Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower left')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 7. Best Model Selection

In [ ]:
# Select best model based on ROC-AUC
best_model_name = results.loc[results['ROC-AUC'].idxmax(), 'Model']

if best_model_name == 'Logistic Regression':
    best_model = lr_model
    best_proba = y_test_proba_lr
elif best_model_name == 'Random Forest':
    best_model = rf_model
    best_proba = y_test_proba_rf
elif best_model_name == 'XGBoost':
    best_model = xgb_model
    best_proba = y_test_proba_xgb
else:
    best_model = xgb_tuned
    best_proba = y_test_proba_tuned

print("="*70)
print("BEST MODEL SELECTED")
print("="*70)
print(f"\n🏆 Model: {best_model_name}")
print(f"\nTest Set Performance:")
print(f"ROC-AUC:   {roc_auc_score(y_test, best_proba):.4f}")
print(f"PR-AUC:    {average_precision_score(y_test, best_proba):.4f}")
print(f"Precision: {precision_score(y_test, best_model.predict(X_test)):.4f}")
print(f"Recall:    {recall_score(y_test, best_model.predict(X_test)):.4f}")

---
## 8. Model Persistence

In [ ]:
# Save best model
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Save best model
best_model_path = models_dir / 'best_model.pkl'
joblib.dump(best_model, best_model_path)
print(f"✓ Best model saved: {best_model_path}")

# Save all models for comparison
joblib.dump(lr_model, models_dir / 'logistic_regression.pkl')
joblib.dump(rf_model, models_dir / 'random_forest.pkl')
joblib.dump(xgb_model, models_dir / 'xgboost.pkl')
if 'xgb_tuned' in locals():
    joblib.dump(xgb_tuned, models_dir / 'xgboost_tuned.pkl')

print("✓ All models saved")

# Save results table
results.to_csv(models_dir / 'model_comparison.csv', index=False)
print(f"✓ Model comparison saved: {models_dir / 'model_comparison.csv'}")

---
## 9. Summary

In [ ]:
print("="*70)
print("MODEL TRAINING SUMMARY")
print("="*70)

print("\n1. MODELS TRAINED:")
print("   - Logistic Regression (Baseline)")
print("   - Random Forest")
print("   - XGBoost")
print("   - XGBoost (Hyperparameter Tuned)")

print("\n2. CLASS IMBALANCE HANDLING:")
print("   - Method: Class weights (balanced)")
print(f"   - Original ratio: {(1-y_train.mean())/y_train.mean():.1f}:1 (Paid:Default)")

print("\n3. BEST MODEL:")
print(f"   - Model: {best_model_name}")
print(f"   - ROC-AUC: {results['ROC-AUC'].max():.4f}")
print(f"   - PR-AUC: {results.loc[results['ROC-AUC'].idxmax(), 'PR-AUC']:.4f}")

print("\n4. MODEL COMPARISON:")
print(results[['Model', 'ROC-AUC', 'PR-AUC', 'Precision', 'Recall']].to_string(index=False))

print("\n5. TOP 10 FEATURES (XGBoost):")
print(feature_importance_xgb.head(10).to_string(index=False))

print("\n6. FILES SAVED:")
print(f"   - Best model: models/best_model.pkl")
print(f"   - All models: models/*.pkl")
print(f"   - Comparison: models/model_comparison.csv")

print("\n" + "="*70)
print("NEXT STEPS - MODEL EVALUATION & INTERPRETATION")
print("="*70)

print("""
1. Business metrics evaluation:
   - Expected Credit Loss (ECL)
   - Cost-sensitive analysis
   - Precision@K (top 10%, 20% riskiest loans)
   - Profit curves under different thresholds

2. Model interpretation:
   - SHAP values (global and local explanations)
   - LIME for individual predictions
   - Partial Dependence Plots (PDP)
   - Feature importance deep dive

3. Model validation:
   - Calibration curves
   - Reliability diagrams
   - Fairness analysis
   - Error analysis
""")

print("="*70)